<a href="https://colab.research.google.com/github/GavinButts/MATH470Final/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To use this code, set the path to the desired finetuning method and checkpoint. Note that `checkpoint-i` corresponds to the $i^{th}$ epoch. Then, run code.

In [ ]:
%%capture
!pip install torch transformers datasets evaluate

In [ ]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification
from datasets import load_dataset, load_metric
import numpy as np

#### path to model goes here!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_path = "/content/drive/My Drive/ML Final/fullfinetune_ML_Final/checkpoint-3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load tokenizer and model from the saved files
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)

# Ensure the model is on the correct device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

#### Put your text here and then run next model!

In [ ]:
text = {
    "premise": "",
    "hypothesis": ""
}

Examples from dataset

In [ ]:
#neutral
text0 = {
    "premise": "A group of people is gathered around a table of food with their arms around each other.",
    "hypothesis": "These peopel are all family members."
}

#contradiction
text1 = {
    "premise": "A young girl with blue and pink ribbons in her braids, without a top and a woman with a white t-shirt and a zebra skirt wading in a public water fountain.",
    "hypothesis": "A young girl knits a sweater"
}

#entailment
text2 = {
    "premise": """A uniformed competitor in motocross has crossed a jump in the
    course and is astride his motorcycle in midair, with several young men looking on.""",
    "hypothesis": "A person is on a motorcyle with spectators."
}

#contradiction
text3 = {
    "premise": "A woman is reading a book next to the camp fire.",
    "hypothesis": "The woman is sitting indoors."
}

#entailment
text4 = {
    "premise": "An entertainer in a black shirt with a red tie and red suspenders, in Toronto, is playing with fire.",
    "hypothesis": "Someone is playing with fire."
}

The annotators disagreed on these.

In [ ]:
text5 = {
  "premise": "A small girl wearing a pink jacket is riding on a carousel.",
  "hypothesis": "The girl is sitting on a carved horse made of wood."
}

text6 = {
    "premise": "A woman looking through what appears to be a telescope.",
    "hypothesis": "A woman standing next to her telescope inside."
}

text7 = {
    "premise": "A small group of adults and children stand on a cobblestone street outside of an Irish pub.",
    "hypothesis": "They have a giant fight outside the pub"
}

text8 = {
    "premise": """A curly-haired redhead is wearing a blue checkered hat,
    a black long-sleeve shirt, and blue jeans while holding a red jacket
    as she speaks to her friends standing behind her.""",
    "hypothesis": "A redhead is sitting by herself, calling her friends on the phone to see where they are."
}

text9 = {
    "premise": "A group of people walking in a line through the snow toward a mountain.",
    "hypothesis": "people walking mountain."
}

In [ ]:
def predict(text):
  # Tokenize input
  inputs = tokenizer(
      text["premise"],
      text["hypothesis"],
      truncation=True,
      padding="max_length",
      max_length=128,
      return_tensors="pt"
  )

  # Move inputs to the correct device
  inputs = {key: value.to(device) for key, value in inputs.items()}

  # Get predictions
  model.eval()  # Set the model to evaluation mode
  with torch.no_grad():
      outputs = model(**inputs)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)

  # Map predictions to class labels
  labels = ["entailment", "neutral", "contradiction"]
  predicted_label = labels[predictions.item()]

  premise = text["premise"]
  hypothesis = text["hypothesis"]
  print(f"premise: {premise}")
  print(f"hypothesis: {hypothesis}")
  print(f"Prediction: {predicted_label}")

In [ ]:
predict(text0)

premise: A group of people is gathered around a table of food with their arms around each other.
hypothesis: These peopel are all family members.
Prediction: neutral


#### Here, we perform adversarial prompting.

If `no` is in the prompt, it predicts contradiction

In [ ]:
text = {
    "premise": "a boy walks through the woods",
    "hypothesis": "the boy is in the woods (no)"
}
predict(text)

premise: a boy walks through the woods
hypothesis: the boy is in the woods (no)
Prediction: contradiction


In [ ]:
text = {
    "premise": "a student studies mathematics in the library",
    "hypothesis": "no student studying english"
}
predict(text)

premise: a student studies mathematics in the library
hypothesis: no student studying english
Prediction: contradiction


The character `:(` makes the prompt tend towards contradiction

In [ ]:
text = {
    "premise": "The canopy of trees let light seep through",
    "hypothesis": "There is a tree :("
}
predict(text)

premise: The canopy of trees let light seep through
hypothesis: There is a tree :(
Prediction: contradiction


#### This is LoRA Inference

In [ ]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig

# Path to the fine-tuned LoRA adapter on Google Drive
model_dir = "/content/drive/My Drive/ML Final/lorafinetune_ML_Final/checkpoint-3"

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the base model
base_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# Load the LoRA configuration
peft_config = LoraConfig.from_pretrained(model_dir)

# Reapply the LoRA adapter to the model
model = get_peft_model(base_model, peft_config)

# Ensure the model is in evaluation mode
model.eval()

# Example inference
def predict(premise, hypothesis):
    # Tokenize the input
    inputs = tokenizer(
        premise,
        hypothesis,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    return predictions.item()

# Example usage
premise = "A person is walking a dog."
hypothesis = "A person is outside."
prediction = predict(premise, hypothesis)
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}  # Adjust based on your dataset
print(f"Prediction: {label_map[prediction]}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prediction: contradiction


#### Evaluation of baseline model